In [9]:
import pandas as pd
import fastf1

In [10]:
fastf1.Cache.enable_cache('cache')
year = 2022
schedule = fastf1.get_event_schedule(year)

In [27]:
# Initialize empty lists to store DataFrames
qualifying = []
races = []

# Define the types of data
race_types=['Q','R']

# Iterate over each row in the schedule
for index, row in schedule.iterrows():
    for race_type in race_types:
        event = row['Location']
        session = fastf1.get_session(year, event, race_type)

        try:
            session.load()
        except:
            print('No data for ' + event)
            continue

        if race_type == 'Q':
            # For qualifying, drop duplicates and keep only the fastest lap per team
            df = session.laps.sort_values(by='LapTime').drop_duplicates(subset='Driver', keep='first')
            df['Position_qualifying'] = df['LapTime'].rank()
            df['Event'] = event
            qualifying.append(df[['Driver', 'Event', 'Position_qualifying']])
        else:
            # For races, sum all laps and calculate the position for each driver
            df = session.laps.groupby('Driver')['LapTime'].sum().reset_index()
            df['Position_race'] = df['LapTime'].rank()
            df['Event'] = event
            races.append(df[['Driver', 'Event', 'Position_race']])

# Concatenate all DataFrames for qualifying and races
total_qualifying = pd.concat(qualifying)
total_races = pd.concat(races)

# Merge qualifying and race DataFrames on 'Driver' and 'Event' columns
total_comparison = pd.merge(total_qualifying, total_races, on=['Driver', 'Event'])

# Now, calculate the position change from qualifying to race
total_comparison['PositionChange'] = total_comparison['Position_race'] - total_comparison['Position_qualifying']

# Print the DataFrame
print(total_comparison[['Driver', 'Event', 'Position_qualifying', 'Position_race', 'PositionChange']])


core           INFO 	Loading data for Spanish Grand Prix - Qualifying [v3.0.3]
req            INFO 	Using cached data for driver_info
logger      WARNING 	Failed to load result data from Ergast!
core        WARNING 	No result data for this session available on Ergast! (This is expected for recent sessions)
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['16', '1', '63', '44', '55', '11', '4', '20', '77', '31', '3', '5', '14', '24', '22', '18', '23', '6', '47', '10']
co

    Driver       Event  Position_qualifying  Position_race  PositionChange
0      LEC       Spain                  1.0            1.0             0.0
1      VER       Spain                  2.0            6.0             4.0
2      SAI       Spain                  3.0           16.0            13.0
3      RUS       Spain                  4.0           11.0             7.0
4      PER       Spain                  5.0            8.0             3.0
..     ...         ...                  ...            ...             ...
472    MAG  Yas Island                 16.0           14.0            -2.0
473    GAS  Yas Island                 17.0            9.0            -8.0
474    BOT  Yas Island                 18.0           10.0            -8.0
475    ALB  Yas Island                 19.0            7.0           -12.0
476    LAT  Yas Island                 20.0            3.0           -17.0

[477 rows x 5 columns]


In [33]:

total_comparison[['Driver', 'Event', 'Position_qualifying', 'Position_race', 'PositionChange']]


,Driver,Event,Position_qualifying,Position_race,PositionChange
0,LEC,Spain,1.0,1.0,0.0
1,VER,Spain,2.0,6.0,4.0
2,SAI,Spain,3.0,16.0,13.0
3,RUS,Spain,4.0,11.0,7.0
4,PER,Spain,5.0,8.0,3.0
...,...,...,...,...,...
472,MAG,Yas Island,16.0,14.0,-2.0
473,GAS,Yas Island,17.0,9.0,-8.0
474,BOT,Yas Island,18.0,10.0,-8.0
475,ALB,Yas Island,19.0,7.0,-12.0


In [34]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Initialize the dictionary for team colors
team_colors = {}

# Function to adjust the color for each driver
def adjust_color(color, amount=0.5):
    hsv = mcolors.rgb_to_hsv(mcolors.to_rgba(color)[:3])
    adjustment = max(0, min(1, hsv[2] * amount))
    hsv[2] = adjustment
    return mcolors.hsv_to_rgb(hsv)

plt.figure(figsize=(10, 5))
for index, (driver, driver_data) in enumerate(total_comparison.groupby('Driver')):
    team = driver_data['Team'].iloc[0]  # Get the team of the driver

    # Get the base color for the team using fastf1, default to black if not found
    base_color = team_colors.get(team, fastf1.plotting.team_color(team))

    color = adjust_color(base_color, 1 + index * 0.1)  # Adjust the color for each driver
    plt.plot(driver_data['GP_Count'], driver_data['Position_qualifying'], color=color, label=f'{driver} Qualifying Position')
    plt.plot(driver_data['GP_Count'], driver_data['Position_race'], color=color, linestyle='--', label=f'{driver} Race Position')

# Updating team_colors with used colors for future reference
team_colors.update({team: base_color for team in total_comparison['Team'].unique() if team not in team_colors})

plt.gca().invert_yaxis()  # Invert the y-axis
plt.xlabel('Grand Prix')
plt.ylabel('Position')
plt.title('Qualifying vs Race Position over Time')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')  # Position the legend outside of the plot
plt.show()


KeyError: 'Team'

<Figure size 1000x500 with 0 Axes>